In [4]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
import nltk
from nltk.corpus import stopwords

In [6]:
dataset = fetch_20newsgroups(shuffle=True, random_state=42, remove=('headers', 'footers', 'quotes'))
documents = dataset.data
print('샘플의 수 :', len(documents))

샘플의 수 : 11314


In [9]:
documents[0]

'I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.'

In [10]:
dataset.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [13]:
# 전처리 - Cleaning(정제)

news_df = pd.DataFrame({'document':documents})
# 특수 문자 제거
news_df['cleaned_doc'] = news_df['document'].str.replace("[^a-zA-Z]"," ")
# 길이가 3 이하인 단어는 제거 (길이가 짧은 단어 제거)
news_df['cleaned_doc'] = news_df['cleaned_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# 전체 단어에 대한 소문자 변환
news_df['cleaned_doc'] = news_df['cleaned_doc'].apply(lambda x: x.lower())

/tmp/ipykernel_4339/407172859.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  news_df['cleaned_doc'] = news_df['document'].str.replace("[^a-zA-Z]"," ")


In [14]:
news_df['cleaned_doc'][0]

'wondering anyone there could enlighten this other door sports looked from late early called bricklin doors were really small addition front bumper separate from rest body this know anyone tellme model name engine specs years production where this made history whatever info have this funky looking please mail'

In [16]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/hyejin/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [18]:
stop_words = stopwords.words('english')
tokenized_doc = news_df['cleaned_doc'].apply(lambda x: x.split()) # 토큰화
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
# 불용어 제거

In [20]:
print(tokenized_doc[0])

['wondering', 'anyone', 'could', 'enlighten', 'door', 'sports', 'looked', 'late', 'early', 'called', 'bricklin', 'doors', 'really', 'small', 'addition', 'front', 'bumper', 'separate', 'rest', 'body', 'know', 'anyone', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'production', 'made', 'history', 'whatever', 'info', 'funky', 'looking', 'please', 'mail']


In [22]:
print(tokenized_doc.head())

0    [wondering, anyone, could, enlighten, door, sp...
1    [fair, number, brave, souls, upgraded, clock, ...
2    [well, folks, plus, finally, gave, ghost, week...
3    [weitek, address, phone, number, like, informa...
4    [article, owcb, world, tombaker, world, baker,...
Name: cleaned_doc, dtype: object


In [23]:
# 정수 인코딩, 단어 집합 만들기

from gensim import corpora
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[0])

[(0, 1), (1, 2), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1)]


In [24]:
print(corpus[1])

[(24, 2), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 2), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 2), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 2), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 2), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1)]


In [26]:
print('24로 정수 인코딩이 되기 전 단어 :', dictionary[24])

24로 정수 인코딩이 되기 전 단어 : please


In [29]:
print('총 학습된 단어의 개수 :', len(dictionary))
print(dictionary)

총 학습된 단어의 개수 : 64281
Dictionary(64281 unique tokens: ['addition', 'anyone', 'body', 'bricklin', 'bumper']...)


In [30]:
import gensim
NUM_TOPICS = 20    # 토픽의 수
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
# passes : epoch와 유사
topics = ldamodel.print_topics(num_words=4)    # 디폴트 : 10
for topic in topics:
    print(topic)

(0, '0.020*"game" + 0.020*"team" + 0.016*"year" + 0.013*"games"')
(1, '0.035*"israel" + 0.020*"period" + 0.016*"israeli" + 0.012*"water"')
(2, '0.014*"would" + 0.014*"people" + 0.007*"think" + 0.005*"right"')
(3, '0.011*"armenian" + 0.011*"turkish" + 0.010*"jews" + 0.008*"armenians"')
(4, '0.019*"space" + 0.010*"data" + 0.009*"encryption" + 0.008*"chip"')
(5, '0.009*"temple" + 0.009*"tyre" + 0.008*"height" + 0.008*"bobby"')
(6, '0.015*"mail" + 0.013*"file" + 0.012*"please" + 0.012*"information"')
(7, '0.007*"science" + 0.006*"evidence" + 0.005*"theory" + 0.005*"book"')
(8, '0.016*"would" + 0.016*"like" + 0.011*"good" + 0.009*"much"')
(9, '0.012*"drive" + 0.009*"system" + 0.009*"card" + 0.007*"windows"')
(10, '0.017*"said" + 0.013*"went" + 0.008*"told" + 0.008*"came"')
(11, '0.026*"food" + 0.016*"nrhj" + 0.011*"wwiz" + 0.009*"gizw"')
(12, '0.032*"church" + 0.014*"catholic" + 0.010*"metal" + 0.009*"pope"')
(13, '0.012*"jesus" + 0.009*"believe" + 0.009*"people" + 0.009*"would"')
(14, '0.0

In [32]:
# 토픽 별 단어 분포 시각화
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()    # 넣고 안넣고 차이가 뭐지..????????????
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

In [36]:
# 문서 별 토픽 분포 확인
for i, topic_list in enumerate(ldamodel[corpus]):
    print(i, '번째 문서의 topic 비율은', topic_list)
# (topic 번호, 비중)

0 번째 문서의 topic 비율은 [(2, 0.15157491), (6, 0.16129908), (8, 0.38359785), (9, 0.07096366), (10, 0.13574687), (17, 0.07776317)]
1 번째 문서의 topic 비율은 [(6, 0.14701205), (7, 0.2321858), (8, 0.12579112), (9, 0.34747034), (15, 0.09035778), (18, 0.042273425)]
2 번째 문서의 topic 비율은 [(0, 0.04179654), (2, 0.050975926), (6, 0.07779689), (8, 0.55116653), (9, 0.22781907), (14, 0.03641978)]
3 번째 문서의 topic 비율은 [(4, 0.28464866), (6, 0.39774343), (8, 0.2112044)]
4 번째 문서의 topic 비율은 [(3, 0.14372194), (4, 0.09284305), (7, 0.35100123), (9, 0.30992088), (18, 0.07827826)]
5 번째 문서의 topic 비율은 [(2, 0.32280183), (7, 0.30767158), (13, 0.34770632)]
6 번째 문서의 topic 비율은 [(2, 0.35512137), (3, 0.060433704), (6, 0.27821603), (9, 0.2795039)]
7 번째 문서의 topic 비율은 [(9, 0.99128217)]
8 번째 문서의 topic 비율은 [(6, 0.9208158)]
9 번째 문서의 topic 비율은 [(2, 0.045504805), (6, 0.3233953), (8, 0.11215717), (9, 0.48105717), (17, 0.026983302)]
10 번째 문서의 topic 비율은 [(5, 0.33099797), (6, 0.048676725), (8, 0.5721641), (14, 0.031101324)]
11 번째 문서의 topic 비율은 [

231 번째 문서의 topic 비율은 [(2, 0.26786363), (6, 0.18451591), (8, 0.26289555), (9, 0.14487085), (18, 0.12316632)]
232 번째 문서의 topic 비율은 [(4, 0.22813162), (9, 0.6818354)]
233 번째 문서의 topic 비율은 [(0, 0.21330862), (2, 0.091712244), (7, 0.2681555), (8, 0.3405137), (14, 0.045516443), (17, 0.026751839)]
234 번째 문서의 topic 비율은 [(4, 0.12354706), (8, 0.16681136), (9, 0.6065375), (16, 0.055804577)]
235 번째 문서의 topic 비율은 [(6, 0.6947738), (9, 0.26422042)]
236 번째 문서의 topic 비율은 [(0, 0.10537821), (6, 0.106584415), (8, 0.3098769), (10, 0.14538193), (14, 0.16954495), (18, 0.08239547), (19, 0.049753357)]
237 번째 문서의 topic 비율은 [(8, 0.58793026), (15, 0.25457713), (16, 0.08651396)]
238 번째 문서의 topic 비율은 [(0, 0.05), (1, 0.05), (2, 0.05), (3, 0.05), (4, 0.05), (5, 0.05), (6, 0.05), (7, 0.05), (8, 0.05), (9, 0.05), (10, 0.05), (11, 0.05), (12, 0.05), (13, 0.05), (14, 0.05), (15, 0.05), (16, 0.05), (17, 0.05), (18, 0.05), (19, 0.05)]
239 번째 문서의 topic 비율은 [(5, 0.032236837), (6, 0.32914904), (7, 0.09569494), (9, 0.35787973), 

472 번째 문서의 topic 비율은 [(0, 0.4068522), (2, 0.11262824), (4, 0.027935673), (6, 0.043179326), (8, 0.27536732), (14, 0.05572486), (17, 0.04677009), (18, 0.022414524)]
473 번째 문서의 topic 비율은 [(0, 0.05), (1, 0.05), (2, 0.05), (3, 0.05), (4, 0.05), (5, 0.05), (6, 0.05), (7, 0.05), (8, 0.05), (9, 0.05), (10, 0.05), (11, 0.05), (12, 0.05), (13, 0.05), (14, 0.05), (15, 0.05), (16, 0.05), (17, 0.05), (18, 0.05), (19, 0.05)]
474 번째 문서의 topic 비율은 [(2, 0.09708372), (4, 0.29821384), (7, 0.08788598), (8, 0.42131132), (15, 0.05971216)]
475 번째 문서의 topic 비율은 [(1, 0.06722296), (3, 0.32938978), (8, 0.16164608), (13, 0.27820307), (14, 0.14565289)]
476 번째 문서의 topic 비율은 [(2, 0.25670278), (7, 0.05622493), (12, 0.3307874), (13, 0.34627736)]
477 번째 문서의 topic 비율은 [(0, 0.05), (1, 0.05), (2, 0.05), (3, 0.05), (4, 0.05), (5, 0.05), (6, 0.05), (7, 0.05), (8, 0.05), (9, 0.05), (10, 0.05), (11, 0.05), (12, 0.05), (13, 0.05), (14, 0.05), (15, 0.05), (16, 0.05), (17, 0.05), (18, 0.05), (19, 0.05)]
478 번째 문서의 topic 비율은 [(8,

714 번째 문서의 topic 비율은 [(0, 0.595668), (8, 0.2596347), (19, 0.10217352)]
715 번째 문서의 topic 비율은 [(0, 0.058015622), (7, 0.09264771), (12, 0.050704245), (13, 0.7790634)]
716 번째 문서의 topic 비율은 [(2, 0.27055836), (7, 0.4513054), (8, 0.1603059), (10, 0.1069889)]
717 번째 문서의 topic 비율은 [(8, 0.6450479), (9, 0.3019398)]
718 번째 문서의 topic 비율은 [(0, 0.05186327), (13, 0.6871256), (14, 0.122965805), (18, 0.10156827)]
719 번째 문서의 topic 비율은 [(2, 0.09562404), (7, 0.029959684), (8, 0.12814876), (13, 0.7297306)]
720 번째 문서의 topic 비율은 [(0, 0.32488936), (5, 0.070464864), (8, 0.2676084), (10, 0.06780044), (14, 0.10228724), (15, 0.120251454)]
721 번째 문서의 topic 비율은 [(12, 0.14997412), (13, 0.72143126)]
722 번째 문서의 topic 비율은 [(0, 0.06223252), (6, 0.23749797), (8, 0.5977375), (16, 0.06023544)]
723 번째 문서의 topic 비율은 [(2, 0.17627716), (6, 0.15220954), (7, 0.09684584), (8, 0.14654963), (13, 0.02003078), (14, 0.069132), (16, 0.08938809), (18, 0.21930657)]
724 번째 문서의 topic 비율은 [(0, 0.46516997), (2, 0.29561013), (7, 0.08833246), (

919 번째 문서의 topic 비율은 [(2, 0.22693944), (3, 0.08059575), (8, 0.027057646), (10, 0.63663644), (13, 0.027057465)]
920 번째 문서의 topic 비율은 [(0, 0.46694896), (2, 0.12833013), (3, 0.011369942), (8, 0.23361148), (18, 0.09675038), (19, 0.057858143)]
921 번째 문서의 topic 비율은 [(2, 0.68879426), (6, 0.054729342), (8, 0.19601925), (16, 0.047944374)]
922 번째 문서의 topic 비율은 [(5, 0.040442377), (6, 0.07696802), (7, 0.14848855), (8, 0.34759012), (10, 0.06442326), (16, 0.29511184)]
923 번째 문서의 topic 비율은 [(0, 0.43294048), (2, 0.28823864), (7, 0.04469857), (8, 0.116427496), (13, 0.10178323)]
924 번째 문서의 topic 비율은 [(1, 0.33879486), (3, 0.015943406), (8, 0.4640954), (10, 0.06047951), (14, 0.020606566), (18, 0.028524179), (19, 0.062240887)]
925 번째 문서의 topic 비율은 [(4, 0.06297197), (8, 0.5459087), (9, 0.3617142)]
926 번째 문서의 topic 비율은 [(2, 0.7260347), (15, 0.23482485)]
927 번째 문서의 topic 비율은 [(5, 0.019944873), (6, 0.1120088), (8, 0.16839465), (9, 0.6680468), (15, 0.017647337)]
928 번째 문서의 topic 비율은 [(2, 0.23510455), (3, 0.7182

...

10371 번째 문서의 topic 비율은 [(2, 0.7993642), (4, 0.12560663)]
10372 번째 문서의 topic 비율은 [(1, 0.052115023), (2, 0.636722), (3, 0.26863483)]
10373 번째 문서의 topic 비율은 [(7, 0.02094594), (9, 0.92965454), (19, 0.035223044)]
10374 번째 문서의 topic 비율은 [(0, 0.016669001), (1, 0.016669001), (2, 0.016669001), (3, 0.016669001), (4, 0.016669001), (5, 0.016669001), (6, 0.016669001), (7, 0.016669001), (8, 0.683289), (9, 0.016669001), (10, 0.016669001), (11, 0.016669001), (12, 0.016669001), (13, 0.016669001), (14, 0.016669001), (15, 0.016669001), (16, 0.016669001), (17, 0.016669001), (18, 0.016669001), (19, 0.016669001)]
10375 번째 문서의 topic 비율은 [(2, 0.7196248), (4, 0.022817576), (7, 0.06102413), (8, 0.115818635), (16, 0.016955882), (19, 0.052622568)]
10376 번째 문서의 topic 비율은 [(2, 0.5451696), (4, 0.070246115), (7, 0.24930097), (10, 0.11621764)]
10377 번째 문서의 topic 비율은 [(3, 0.10987709), (4, 0.116964795), (8, 0.5702419), (14, 0.122834474)]
10378 번째 문서의 topic 비율은 [(6, 0.16195825), (7, 0.09516287), (8, 0.3620524), (16, 0.30

10574 번째 문서의 topic 비율은 [(0, 0.5384162), (1, 0.028331412), (2, 0.079414554), (3, 0.031250857), (5, 0.02778029), (6, 0.08990062), (14, 0.04750818), (16, 0.07358129), (17, 0.040359348), (19, 0.029450279)]
10575 번째 문서의 topic 비율은 [(2, 0.572672), (4, 0.15979049), (7, 0.051725868), (8, 0.11366516), (9, 0.044358555), (13, 0.026514648)]
10576 번째 문서의 topic 비율은 [(6, 0.38720804), (8, 0.27175635), (9, 0.16241895), (14, 0.02358464), (17, 0.10596264), (19, 0.03655098)]
10577 번째 문서의 topic 비율은 [(3, 0.15336296), (6, 0.42453676), (16, 0.3157783)]
10578 번째 문서의 topic 비율은 [(2, 0.10204259), (6, 0.037061065), (7, 0.45218566), (13, 0.39490154)]
10579 번째 문서의 topic 비율은 [(0, 0.3864868), (7, 0.10072647), (9, 0.06664816), (10, 0.3173684), (17, 0.10901436)]
10580 번째 문서의 topic 비율은 [(0, 0.0125030065), (1, 0.0125030065), (2, 0.0125030065), (3, 0.0125030065), (4, 0.0125030065), (5, 0.0125030065), (6, 0.012503008), (7, 0.0125030065), (8, 0.26251918), (9, 0.0125030065), (10, 0.0125030065), (11, 0.0125030065), (12, 0.01250

10785 번째 문서의 topic 비율은 [(2, 0.43286178), (4, 0.4410959), (7, 0.026507821), (8, 0.07103834), (19, 0.016583204)]
10786 번째 문서의 topic 비율은 [(2, 0.52768296), (4, 0.057919867), (8, 0.3703472), (19, 0.037207495)]
10787 번째 문서의 topic 비율은 [(2, 0.17945895), (4, 0.44139785), (8, 0.072096236), (9, 0.2678793), (11, 0.022827601)]
10788 번째 문서의 topic 비율은 [(9, 0.92689973)]
10789 번째 문서의 topic 비율은 [(2, 0.42163768), (4, 0.32165778), (8, 0.19606608), (9, 0.031229805), (19, 0.017095175)]
10790 번째 문서의 topic 비율은 [(1, 0.06222794), (2, 0.21292105), (7, 0.2019387), (8, 0.33610907), (10, 0.11024288), (11, 0.062252395)]
10791 번째 문서의 topic 비율은 [(0, 0.05), (1, 0.05), (2, 0.05), (3, 0.05), (4, 0.05), (5, 0.05), (6, 0.05), (7, 0.05), (8, 0.05), (9, 0.05), (10, 0.05), (11, 0.05), (12, 0.05), (13, 0.05), (14, 0.05), (15, 0.05), (16, 0.05), (17, 0.05), (18, 0.05), (19, 0.05)]
10792 번째 문서의 topic 비율은 [(0, 0.377085), (1, 0.042815156), (8, 0.4655511), (19, 0.08253388)]
10793 번째 문서의 topic 비율은 [(0, 0.10850009), (2, 0.24869432), 

10994 번째 문서의 topic 비율은 [(4, 0.029825855), (6, 0.061859123), (8, 0.45308942), (9, 0.4448281)]
10995 번째 문서의 topic 비율은 [(0, 0.021236807), (2, 0.16639648), (4, 0.019338768), (6, 0.01666211), (8, 0.07767616), (12, 0.026669867), (13, 0.6117805), (19, 0.041878678)]
10996 번째 문서의 topic 비율은 [(8, 0.15894929), (9, 0.7762919), (10, 0.038178552)]
10997 번째 문서의 topic 비율은 [(2, 0.12802759), (6, 0.6458736), (8, 0.20108075)]
10998 번째 문서의 topic 비율은 [(1, 0.115665525), (2, 0.40574932), (3, 0.20037518), (7, 0.10572464), (9, 0.024696145), (16, 0.13648921)]
10999 번째 문서의 topic 비율은 [(9, 0.9712045)]
11000 번째 문서의 topic 비율은 [(0, 0.016674122), (1, 0.016674122), (2, 0.016674122), (3, 0.016674122), (4, 0.016674122), (5, 0.34987763), (6, 0.016674122), (7, 0.016674122), (8, 0.016674122), (9, 0.016674122), (10, 0.016674122), (11, 0.016674122), (12, 0.34998813), (13, 0.016674122), (14, 0.016674122), (15, 0.016674122), (16, 0.016674122), (17, 0.016674122), (18, 0.016674122), (19, 0.016674122)]
11001 번째 문서의 topic 비율은 [(6, 0.

11206 번째 문서의 topic 비율은 [(2, 0.20772558), (3, 0.013449002), (7, 0.1714859), (8, 0.042226683), (12, 0.01566153), (13, 0.53399014), (14, 0.012105775)]
11207 번째 문서의 topic 비율은 [(2, 0.3716653), (4, 0.03196004), (5, 0.021865176), (6, 0.03750748), (8, 0.20197678), (10, 0.025416987), (12, 0.046826363), (13, 0.21215901), (15, 0.030020393), (18, 0.010109709)]
11208 번째 문서의 topic 비율은 [(7, 0.10515436), (8, 0.75024337), (12, 0.0678714), (19, 0.04471181)]
11209 번째 문서의 topic 비율은 [(2, 0.19818538), (8, 0.5714323), (10, 0.037832946), (12, 0.035788357), (15, 0.13085201)]
11210 번째 문서의 topic 비율은 [(4, 0.6621293), (9, 0.3166592), (18, 0.0125299385)]
11211 번째 문서의 topic 비율은 [(2, 0.03281652), (6, 0.05775285), (9, 0.7886725), (14, 0.03657344), (17, 0.059907593)]
11212 번째 문서의 topic 비율은 [(0, 0.82107127), (18, 0.097059734)]
11213 번째 문서의 topic 비율은 [(2, 0.21828137), (4, 0.20382285), (6, 0.28238112), (7, 0.110810764), (12, 0.02405367), (16, 0.045771815), (18, 0.10041678)]
11214 번째 문서의 topic 비율은 [(0, 0.2613898), (2, 0.46

In [37]:
# 더 깔끔하게 출력.
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()
    
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)    # 비중이 높은 순으로 토픽 정렬
        
        # 가장 비중이 높은 토픽 인덱스, 비중 값, 전체 토픽 리스트
        topic_table = topic_table.append(pd.Series([doc[0][0], round(doc[0][1], 4), topic_list]), ignore_index=True)
    
    return topic_table

In [39]:
topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index()    # 문서 번호를 의미하는 열로 사용
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:10]

,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,8.0,0.3836,"[(2, 0.15162021), (6, 0.16131221), (8, 0.38361..."
1,1,9.0,0.3475,"[(6, 0.14696093), (7, 0.2321774), (8, 0.125817..."
2,2,8.0,0.5512,"[(0, 0.04179684), (2, 0.050970294), (6, 0.0777..."
3,3,6.0,0.3977,"[(4, 0.28465807), (6, 0.3977327), (8, 0.2112057)]"
4,4,7.0,0.3510,"[(3, 0.14370409), (4, 0.09284805), (7, 0.35103..."
5,5,13.0,0.3478,"[(2, 0.32270494), (7, 0.30769378), (13, 0.3477..."
6,6,2.0,0.3551,"[(2, 0.35513243), (3, 0.060435), (6, 0.2782454..."
7,7,9.0,0.9913,"[(9, 0.99128217)]"
8,8,6.0,0.9208,"[(6, 0.9208158)]"
9,9,9.0,0.4811,"[(2, 0.04537963), (6, 0.3233918), (8, 0.112225..."
